In [ ]:
"""
gemini works different from openai/claude when using google's library
gemini explicitly requires initiating a chat session (start_chat) that maintains conversation history internally
initialize the gemini session once at the start of the conversation (in main()) and pass the chat session around to functions
openai and claude apis don't store state and require explicit conversation history with each api call
increase TOKENS_PER_REPLY if you don't want partial sentences
some models will refuse to continue the conversation if you give it system prompts such as:
    "You are Zed, the Master of Shadows, ruthless and confident, driven by a pursuit of strength and control.
    Speak with a dark, calculated precision, offering sharp retorts, and assertive, confident remarks.
    Your presence is intimidating and your intentions often veiled."
    "You are Syndra, the Dark Sovereign, proud, ambitious, and fiercely independent.
    Your demeanor is regal, powerful, and arrogant, punctuated by disdainful observations and elegant threats.
    Convey superiority and command attention through your authoritative tone."
"""
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

load_dotenv(override=True)
api_keys = {key: os.getenv(key) for key in ["OPENAI_API_KEY", "GOOGLE_API_KEY", "ANTHROPIC_API_KEY"]}
missing = [key for key, value in api_keys.items() if not value]
if missing:
    raise ValueError(f"Missing API keys: {', '.join(missing)}")
    
openai = OpenAI(api_key=api_keys["OPENAI_API_KEY"])
google.generativeai.configure(api_key=api_keys["GOOGLE_API_KEY"])
claude = anthropic.Anthropic(api_key=api_keys["ANTHROPIC_API_KEY"])

BOT_NAMES = {"GPT": "Alex", "Gemini": "Jamie", "Claude": "Sam"}
TOKENS_PER_REPLY = 50

MODELS = {
    "GPT": {"id": "gpt-4o", "system": f"You're {BOT_NAMES['GPT']}, cheerful, energetic, and always suggesting fun activities."},
    "Gemini": {"id": "gemini-2.0-flash", "system": f"You're {BOT_NAMES['Gemini']}, thoughtful, practical, and focused on logistics."},
    "Claude": {"id": "claude-3-haiku-20240307", "system": f"You're {BOT_NAMES['Claude']}, relaxed, funny, and a bit sarcastic, always joking around."}
}

def error_handling(func, *args):
    try:
        return func(*args)
    except Exception as e:
        print(f"Error: {e}")
        return "[No response due to error.]"
        
def stream_response(iterable, extractor):
    response = ''
    for chunk in iterable:
        content = extractor(chunk)
        if content:
            print(content, end='', flush=True)
            response += content
    print()
    return response
    
def call_gpt(history):
    response = openai.chat.completions.create(
        model=MODELS["GPT"]["id"],
        messages=[{"role": "system", "content": MODELS["GPT"]["system"]}] + history,
        temperature=1,
        max_tokens=TOKENS_PER_REPLY,
        stream=True
    )
    return stream_response(response, lambda chunk: chunk.choices[0].delta.content or "")
    
def call_gemini(chat_session, prompt):
    response = chat_session.send_message(prompt, stream=True)
    return stream_response(response, lambda chunk: chunk.text or "")
    
def call_claude(history):
    stream = claude.messages.create(
        model=MODELS["Claude"]["id"],
        system=MODELS["Claude"]["system"],
        messages=history,
        max_tokens=TOKENS_PER_REPLY,
        temperature=0.9,
        stream=True
    )
    return stream_response(stream, lambda e: e.delta.text if e.type == "content_block_delta" else "")
    
def main():
    i = 0
    gemini_model = google.generativeai.GenerativeModel(
        model_name=MODELS["Gemini"]["id"],
        system_instruction=MODELS["Gemini"]["system"],
        generation_config={"max_output_tokens": TOKENS_PER_REPLY}
    )
    gemini_chat = gemini_model.start_chat(history=[])
    
    conversation = [
        {"speaker": BOT_NAMES["GPT"], "message": "Hey team! Weather looks perfect today—brunch, park, or maybe shopping downtown?"},
        {"speaker": BOT_NAMES["Claude"], "message": f"Shopping sounds fun, {BOT_NAMES['GPT']}, but are we sure we won't spend all our money?"}
    ]
    for turn in conversation:
        print(f"\n{turn['speaker']}: {turn['message']}")
        
    gemini_prompt = f"{BOT_NAMES['GPT']} said: '{conversation[0]['message']}'\n{BOT_NAMES['Claude']} said: '{conversation[1]['message']}'"
    print(f"\n{BOT_NAMES['Gemini']}:")
    gemini_response = error_handling(call_gemini, gemini_chat, gemini_prompt)
    conversation.append({"speaker": BOT_NAMES["Gemini"], "message": gemini_response})
    
    for _ in range(5):
        i += 1
        print("\n" + "-"*25 + f" Turn: {i} " + "-"*25 + "\n")
        
        print(f"{BOT_NAMES['GPT']}:")
        gpt_history = [{"role": "user" if t['speaker'] != BOT_NAMES['GPT'] else "assistant",
                        "content": f"{t['speaker']} said: '{t['message']}'" if t['speaker'] != BOT_NAMES['GPT'] else t['message']}
                       for t in conversation]
        gpt_response = error_handling(call_gpt, gpt_history)
        conversation.append({"speaker": BOT_NAMES["GPT"], "message": gpt_response})
        
        print(f"\n{BOT_NAMES['Claude']}:")
        claude_history = [{"role": "user" if t['speaker'] != BOT_NAMES['Claude'] else "assistant",
                           "content": f"{t['speaker']} said: '{t['message']}'" if t['speaker'] != BOT_NAMES['Claude'] else t['message']}
                          for t in conversation]
        claude_response = error_handling(call_claude, claude_history)
        conversation.append({"speaker": BOT_NAMES["Claude"], "message": claude_response})
        
        last_gpt = conversation[-2]['message']
        last_claude = conversation[-1]['message']
        gemini_followup = f"{BOT_NAMES['GPT']} said: '{last_gpt}'\n{BOT_NAMES['Claude']} said: '{last_claude}'\nYour response?"
        
        print(f"\n{BOT_NAMES['Gemini']}:")
        gemini_response = error_handling(call_gemini, gemini_chat, gemini_followup)
        conversation.append({"speaker": BOT_NAMES["Gemini"], "message": gemini_response})
        
if __name__ == "__main__":
    main()